In [1]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-248a6346-56fc-4fbc-9dcf-e317fe96fb90;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (41ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (32ms)
:: resolution report :: resolve 654ms :: artifacts dl 75ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/25 04:48:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 04:48:03 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
%pip install tensorflow --quiet
%pip install tensorflow_gnn --quiet
%pip install tensorflow_io --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.0 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from tensorflow.keras import layers
from tensorflow import feature_column
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

2023-03-25 04:49:41.708560: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 04:49:42.868712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_vi

In [89]:
# https://stackoverflow.com/questions/68036975/valueerror-shape-must-be-at-least-rank-3-but-is-rank-2-for-node-biasadd
# config for rank error in lstm
tf.config.threading.set_inter_op_parallelism_threads(16)
pd.set_option('display.max_columns', None)
tf.keras.backend.set_image_data_format("channels_last")

In [105]:
# Set Config
embedding_dim = 64
max_length = 6
padding_type = 'post'
trunc_type = 'post'
training_portion = 0.8

In [5]:
df = spark.read.parquet("s3a://sapient-bucket-trusted/prod/tensor_sample_data/test_holdout/*").cache().toPandas()

In [6]:
len(df)

2598111

In [7]:
df.dtypes

Trace              int64
malicious          int32
event_sequence    object
dtype: object

In [77]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [78]:
x = df['event_sequence']

In [79]:
x

0          [1000111000000111001000001, 111001001111100110...
1          [1111011111100100111000000, 001001100010110101...
2          [0011010000010001001100010, 011001110100101000...
3          [1010100011110001011000011, 100111000101000101...
4          [0000111001010100000000101, 101011110111101010...
                                 ...                        
2598106    [0010111101100111111000010, 000100010110101001...
2598107    [0000011101110111111110110, 001001011011100001...
2598108    [0100100011000101111000001, 001110100010100110...
2598109    [0001101110101110111101110, 001100001001100111...
2598110    [0000001101011000010011100, 000110000010101010...
Name: event_sequence, Length: 2598111, dtype: object

In [80]:
tokenizer.fit_on_texts(x)

In [94]:
train_sequences = tokenizer.texts_to_sequences(x)

In [95]:
df['event_seq'] = train_sequences

In [83]:
df.head(2)

,Trace,malicious,event_sequence,event_seq
0,111669149697,0,"[1000111000000111001000001, 111001001111100110...","[2677810, 2677811, 2677812, 2677813, 2677814, ..."
1,111669149698,0,"[1111011111100100111000000, 001001100010110101...","[2677815, 2677816, 398042, 2677817, 2677818, 3..."


In [84]:
df.dtypes

Trace              int64
malicious          int32
event_sequence    object
event_seq         object
dtype: object

In [11]:
len(df.event_sequence.tolist()[0])

6

In [96]:
print(train_sequences[10])

[2677851, 2677852, 2677853, 2677854, 398055, 398056]


In [113]:
training_data = df['event_sequence'].tolist()

In [114]:
training_label = df['malicious'].tolist()

In [115]:
train_size = int(len(training_label) * training_portion)

train_set = training_data[0: train_size]
train_labels = training_label[0: train_size]
validation_set = training_data[train_size:]
validation_labels = training_label[train_size:]

In [116]:
train_sequences = tokenizer.texts_to_sequences(train_set)
train_padded = tf.keras.utils.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [118]:
validation_sequences = tokenizer.texts_to_sequences(validation_set)
validation_padded = tf.keras.utils.pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [124]:
train_ds = tf.ragged.constant(train_labels)
validation_ds = tf.ragged.constant(validation_labels)

In [119]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])

2023-03-25 07:19:22.136592: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-25 07:19:22.137817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-25 07:19:22.139142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [120]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          159966912 
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 6)                 390       
                                                                 
Total params: 160,041,606
Trainable params: 160,041,606
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [125]:
num_epochs = 10
history = model.fit(train_padded, train_ds, epochs=num_epochs, 
                    validation_data=(validation_padded, validation_ds), 
                    verbose=2)

Epoch 1/10


2023-03-25 07:24:27.850460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-25 07:24:27.852131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-25 07:24:27.853294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_3/bidirectional_2/forward_lstm_2/PartitionedCall]] [Op:__inference_train_function_9280]

In [126]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: name 'history' is not defined

In [12]:
# ds = tf.data.Dataset.from_tensor_slices(([i for i in df['event_sequence']], df['Trace'], df['malicious']))

In [13]:
# Input data based on - https://www.tensorflow.org/tutorials/structured_data/feature_columns
train, test = train_test_split(df, test_size=0.2)

In [14]:
train, val = train_test_split(train, test_size=0.2)

In [15]:
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1662790 train examples
415698 validation examples
519623 test examples


In [16]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('malicious')
  ds = tf.data.Dataset.from_tensor_slices(([i for i in df['event_sequence']], df['malicious']))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [17]:
# A utility method to create a feature column
# and to transform a batch of data
def make_features(feature_column):  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [22]:
example_batch = next(iter(train))

In [23]:
example_batch

'Trace'

In [24]:
vocab_size = len(set([x[0] for x in df['event_sequence']] ))

In [25]:
# Vocabulary size and number of words in a sequence.
vocab_size = vocab_size
sequence_length = 6

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [26]:
text_ds = train_ds.map(lambda x, y: x)

NameError: name 'train_ds' is not defined

In [119]:
vectorize_layer.adapt(tf.reshape(text_ds, (len(text_ds),6,1)))

ValueError: Attempt to convert a value (<_MapDataset element_spec=TensorSpec(shape=(None, 6), dtype=tf.string, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.map_op._MapDataset'>) to a Tensor.

In [28]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [29]:
for sequence_batch, label_batch in train_ds.take(1):
  print('A batch of sequences:', sequence_batch ) # list(feature_batch.keys()))
  print('A batch of targets:', label_batch )

2023-03-25 04:56:48.184864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2598111,6]
	 [[{{node Placeholder/_0}}]]
2023-03-25 04:56:48.185197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [2598111]
	 [[{{node Placeholder/_1}}]]


A batch of sequences: tf.Tensor(
[[b'1001111100101101011011110' b'0110010010011000100010111'
  b'0101000010110001010100101' b'1100111000000110111110101'
  b'1100010111011011100110001' b'0110110111000100110110101']
 [b'1100001010001101001101011' b'0000101111111110111101001'
  b'0000010101110110110010010' b'0000111101110011001111001'
  b'0000001100000101001000101' b'1010011100110110000111101']
 [b'0000011001111010101100011' b'0101010101001100110110010'
  b'0111100111100001000000001' b'1101010111101111010011000'
  b'1100111000111111101001011' b'1101111110000100111011000']
 [b'1110011001111100011001111' b'1000101000111010100011000'
  b'0000101101100111100110111' b'0111111111011100011101100'
  b'0110000100110101011000001' b'1011110100010100101111000']
 [b'0001010110111100000011111' b'0111010111011100000001101'
  b'1100011100000010101100100' b'1001010011101110000011101'
  b'1110011011001010111110101' b'1000111100000101001101010']], shape=(5, 6), dtype=string)
A batch of targets: tf.Tensor([0

In [32]:
model = tf.keras.Sequential([
  #feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

In [33]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [34]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10


2023-03-25 04:57:28.560300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2598111,6]
	 [[{{node Placeholder/_0}}]]
2023-03-25 04:57:28.560604: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [2598111]
	 [[{{node Placeholder/_1}}]]
2023-03-25 04:57:29.556272: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'sequential/dense/Cast' defined at (most recent call last):
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 724, in start
      self.io_loop.start()
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self

In [35]:
data = tf.ragged.constant([ [[940, 203, 668, 387, 790, 320, 939, 185],[315, 515, 791, 181, 939, 787]], 
                             [[564, 205], [820, 180, 993, 739]] ]) 

In [36]:
# https://stackoverflow.com/questions/70399367/sparse-feature-vector-sequence-as-input-to-lstm
X = keras.Input(shape=[inputs.shape[0], inputs.shape[1]], dtype=tf.int64, ragged=True)
l1 = keras.layers.Embedding(inputs.shape[0], 16)(X)
l2 = tf.reduce_sum(l1, axis=2) #To calculate the dense feature vector for each timestep.
l3 = keras.layers.LSTM(32, use_bias=False)(l2)
l4 = keras.layers.Dense(32, activation='sigmoid')(l3)
l5 = keras.layers.Activation(tf.nn.relu)(l4)
output = tf.keras.layers.Dense(1)(l5)

NameError: name 'inputs' is not defined

In [37]:
# Pad the sequences so they are all the same length
max_sequence_length = max([len(seq) for seq in sequences])
total_words = len(tokenizer.word_index) + 1
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

NameError: name 'sequences' is not defined

In [ ]:
print(model(data))

In [ ]:
model.predict(data)

In [10]:
##Building input data
test01 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],'label1')
test02 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[0,0,0,0]],'label2')
test03 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[0,0,0,0],[1,1,1,1]],'label3')
test04 = ([[1,1,1,1],[0,0,0,0],[1,1,1,1],[1,1,1,1],[1,1,1,1]],'label1')
test_data = [test01,test02,test03,test04]


##DataFrame from data
columns = ['feature','label']
t_df = pd.DataFrame(data = test_data, columns = columns)

In [11]:
t_df

,feature,label
0,"[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1,...",label1
1,"[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1,...",label2
2,"[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [0,...",label3
3,"[[1, 1, 1, 1], [0, 0, 0, 0], [1, 1, 1, 1], [1,...",label1


In [12]:
dataset = tf.data.Dataset.from_tensor_slices(([i for i in t_df['feature']], t_df['label']))

In [13]:
dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(5, 4), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [29]:
# https://stackoverflow.com/questions/70399367/sparse-feature-vector-sequence-as-input-to-lstm
X = keras.Input(shape=[t_df.shape[0], t_df.shape[1]], dtype=tf.int64, ragged=True)
l1 = keras.layers.Embedding(t_df.shape[0], 16)(dataset)
l2 = tf.reduce_sum(l1, axis=2) #To calculate the dense feature vector for each timestep.
l3 = keras.layers.LSTM(32, use_bias=False)(l2)
l4 = keras.layers.Dense(32, activation='sigmoid')(l3)
l5 = keras.layers.Activation(tf.nn.relu)(l4)
output = tf.keras.layers.Dense(1)(l5)

AttributeError: Exception encountered when calling layer 'embedding' (type Embedding).

'_TensorSliceDataset' object has no attribute 'dtype'

Call arguments received by layer 'embedding' (type Embedding):
  • inputs=<_TensorSliceDataset element_spec=(TensorSpec(shape=(5, 4), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [27]:
t_df.shape

(4, 2)